## Load 'jupyter notebook' from Conda Terminal before beginning to use GPU

In [1]:
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.optimizers import SGD
import numpy as np
import tensorflow as tf

# Testing to make sure TensorFlow GPU is working
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print('Tensorflow: ', tf.__version__)

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18200490774270905730
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3805675520
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2878038019961921597
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
Tensorflow:  1.11.0


### Load data, shuffle data, normalize data, split data into training and validation

In [2]:
from sklearn.utils import shuffle

# Load the data
X_train = np.load("X_train_3.dat")
y_train = np.load("y_train_3.dat")


X_train, y_train = shuffle(X_train, y_train, random_state=42)
# reshape so in form for CNN-Keras
#X_train = X_train.reshape(X_train.shape[0], 174, 124, 1)

# Normalize the data
X_train = X_train/255

print(X_train.shape)
print(y_train.shape)

# split data into validation set
training_set_size = int(X_train.shape[0] * .80)
X_train_validation = X_train[training_set_size:, :,:,:]
y_train_validation = y_train[training_set_size:, :]
y_train = y_train[:training_set_size,:]
X_train = X_train[:training_set_size, :, :, :]
print("Validation x train set:" + str(X_train_validation.shape))
print("X train set:" + str(X_train.shape))
print("Validation y train set:" + str(y_train_validation.shape))
print("Y train set:" + str(y_train.shape))

(9000, 124, 174, 3)
(9000, 10)
Validation x train set:(1800, 124, 174, 3)
X train set:(7200, 124, 174, 3)
Validation y train set:(1800, 10)
Y train set:(7200, 10)


## Build the CNN network

In [3]:
# VGG-16 like network from Andrew Ng's course on Coursera

model = Sequential()

# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(64, (7,7), activation = 'relu', strides=(1,1), input_shape=X_train.shape[1:]))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

model.add(Conv2D(128, (7, 7), activation = 'relu', strides=(1,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

model.add(Conv2D(256, (7, 7), activation = 'relu', strides=(1,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

model.add(Conv2D(512, (7, 7), activation = 'relu', strides=(1,1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

# model.add(Conv2D(256, (7, 7), activation = 'relu', strides=(1,1)))
# # # model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))
# # # model.add(Dropout(0.5))

# # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [ ]:
# model = Sequential()
# model.add(ZeroPadding2D((1,1),input_shape=(3,124,174)))
# model.add(Convolution2D(64, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(64, 3, 3, activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(128, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(128, 3, 3, activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(256, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(256, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(256, 3, 3, activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, 3, 3, activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, 3, 3, activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(Flatten())
# model.add(Dense(4096, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(4096, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1000, activation='softmax'))

## Begin training the model

In [ ]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

mcp = ModelCheckpoint("models/best_model_3splits_50epochs_highLR", monitor='val_acc', verbose=0, 
                      save_best_only=True, save_weights_only=False, mode='max', period=1)

# adam = Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0, amsgrad=False)
sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(sgd, 'categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=128, epochs=300 , validation_data=(X_train_validation, y_train_validation), 
         callbacks = [mcp])

Train on 7200 samples, validate on 1800 samples
Epoch 1/150
7200/7200 [==============================] - 39s 5ms/step - loss: 2.2292 - acc: 0.2375 - val_loss: 1.8267 - val_acc: 0.3517
Epoch 2/150
7200/7200 [==============================] - 37s 5ms/step - loss: 1.8838 - acc: 0.2954 - val_loss: 1.7133 - val_acc: 0.3661
Epoch 3/150
7200/7200 [==============================] - 37s 5ms/step - loss: 1.8295 - acc: 0.3207 - val_loss: 1.6631 - val_acc: 0.3794
Epoch 4/150
7200/7200 [==============================] - 37s 5ms/step - loss: 1.7499 - acc: 0.3407 - val_loss: 1.5392 - val_acc: 0.4506
Epoch 5/150
7200/7200 [==============================] - 37s 5ms/step - loss: 1.7425 - acc: 0.3531 - val_loss: 1.6086 - val_acc: 0.4061
Epoch 6/150
7200/7200 [==============================] - 37s 5ms/step - loss: 1.6939 - acc: 0.3667 - val_loss: 1.4946 - val_acc: 0.4806
Epoch 7/150
7200/7200 [==============================] - 37s 5ms/step - loss: 1.6560 - acc: 0.3825 - val_loss: 1.5886 - val_acc: 0.4111


In [5]:
model.save("models/trained_music_classifier.h5")

## Evaluate the model

In [57]:
score = model.evaluate(X_train_validation, y_train_validation, batch_size=32, verbose=1)
print(score)

50/50 [==============================] - 0s 1ms/step
[3.419465198516846, 0.5800000071525574]
